In [1]:
import tensorflow as tf
import numpy as np
from tabulate import tabulate
from tensorflow.python.ops.rnn_cell import LSTMCell, GRUCell
from PhasedLSTMCell import PhasedLSTMCell, multiPLSTM

In [2]:
flags = tf.flags
flags.DEFINE_string("unit", "PLSTM", "Can be PSLTM, LSTM, GRU")
flags.DEFINE_integer("n_hidden", 100, "hidden units in the recurrent layer")
flags.DEFINE_integer("n_epochs", 30, "number of epochs")
flags.DEFINE_integer("batch_size", 32, "batch size")
flags.DEFINE_integer("b_per_epoch", 80, "batches per epoch")
flags.DEFINE_integer("n_layers", 4, "hidden units in the recurrent layer")
flags.DEFINE_float("exp_init", 3., "Value for initialization of Tau")
FLAGS = flags.FLAGS

In [3]:
# Net Params
n_input = 1
n_out = 2

In [4]:
def RNN(_X, _weights, _biases, lens):
    if FLAGS.unit == "PLSTM":
        cell = PhasedLSTMCell(FLAGS.n_hidden, use_peepholes=True, state_is_tuple=True)
    elif FLAGS.unit == "GRU":
        cell = GRUCell(FLAGS.n_hidden)
    elif FLAGS.unit == "LSTM":
        cell = LSTMCell(FLAGS.n_hidden, use_peepholes=True, state_is_tuple=True)
    else:
        raise ValueError("Unit '{}' not implemented.".format(FLAGS.unit))
    initial_states = [tf.nn.rnn_cell.LSTMStateTuple(tf.zeros([FLAGS.batch_size, FLAGS.n_hidden], tf.float32), tf.zeros([FLAGS.batch_size, FLAGS.n_hidden], tf.float32)) for _ in range(FLAGS.n_layers)]
    outputs, initial_states = multiPLSTM(_X, FLAGS.batch_size, lens, FLAGS.n_layers, FLAGS.n_hidden, n_input, initial_states)

    outputs = tf.slice(outputs, [0, 0, 0], [-1, -1, FLAGS.n_hidden])
    
    batch_size = tf.shape(outputs)[0]
    max_len = tf.shape(outputs)[1]
    out_size = int(outputs.get_shape()[2])
    index = tf.range(0, batch_size) * max_len + (lens - 1)
    flat = tf.reshape(outputs, [-1, out_size])
    relevant = tf.gather(flat, index)

    return tf.nn.bias_add(tf.matmul(relevant, _weights['out']), _biases['out']), initial_states

#### Building the PLSTM model

In [5]:
x = tf.placeholder(tf.float32, [None, None, n_input + 1])
lens = tf.placeholder(tf.int32, [None])
#labels
y = tf.placeholder(tf.float32, [None, 2])
# weights from input to hidden
weights = {
    'out': tf.Variable(tf.random_normal([FLAGS.n_hidden, n_out], dtype=tf.float32))
}

biases = {
    'out': tf.Variable(tf.random_normal([n_out], dtype=tf.float32))
}

# Register weights to be monitored by tensorboard
w_out_hist = tf.summary.histogram("weights_out", weights['out'])
b_out_hist = tf.summary.histogram("biases_out", biases['out'])
print ("Compiling RNN...",)
predictions, initial_states = RNN(x, weights, biases, lens)
print ("DONE!")
# Register initial_states to be monitored by tensorboard
initial_states_hist = tf.summary.histogram("initial_states", initial_states[0][0])

('Compiling RNN...',)
DONE!


In [7]:
print ("Compiling cost functions...",)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(predictions, y))
print ("DONE!")
cost_summary = tf.summary.scalar("cost", cost)
optimizer = tf.train.AdamOptimizer().minimize(cost)

('Compiling cost functions...',)
DONE!


/Users/abhishek/anaconda/envs/tflow/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [38]:
# evaluation
correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

accuracy_summary = tf.summary.scalar("accuracy", accuracy)

In [53]:
def create_random_batch(batch_size):
    file_index = np.random.choice(range(1, 1001), size=batch_size)
    file_dir = '../data/classify/train/'
    file_values = []
    len_values = []
    Y = np.zeros((batch_size, n_out))
    for j in range(len(file_index)):
        out = np.loadtxt(file_dir + str(file_index[j]) + '.csv', delimiter=',')
        file_values.append(out)
        len_values.append(len(out))
        if file_index[j] > 500:
            Y[j, 1] = 1
        else:
            Y[j, 0] = 1
    max_len = np.max(len_values)
    X = np.zeros((batch_size, max_len, n_input+1))
    for i in range(len(file_values)):
        X[i, max_len-len_values[i]:, :] = np.array(file_values[i])
        
    return X, Y, np.array(len_values)

In [54]:
init = tf.global_variables_initializer()
print ("Initializing variables...",)
sess = tf.Session()
sess.run(init)

('Initializing variables...',)


In [55]:
from tqdm import tqdm

In [56]:
writer = tf.summary.FileWriter("/tmp/run/1", sess.graph)

In [ ]:
for step in range(FLAGS.n_epochs):
    train_cost = 0
    train_acc = 0
    for i in tqdm(range(FLAGS.b_per_epoch)):
        batch_xs, batch_ys, leng = create_random_batch(FLAGS.batch_size)
        res = sess.run([optimizer, cost, accuracy],
                       feed_dict={x: batch_xs,
                                  y: batch_ys,
                                  lens: leng
                                  })
        train_cost += res[1] / FLAGS.b_per_epoch
        train_acc += res[2] / FLAGS.b_per_epoch
    print "Epoch "+ str(i+1) +" train_cost: "+str(train_cost)+" train_accuracy: "+str(train_acc)

  2%|▎         | 2/80 [00:59<39:26, 30.33s/it]